In [62]:
import tifffile as tf
import zarr
import dask.array as da
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import cellcutter
import cellcutter.cli
from numcodecs import Blosc

# import closest
# import forcelayout as fl
# import gridify


In [63]:




# image_path = "/mnt/c/Users/simon/Documents/Greg/for_simon/tif/WD-76845-097.ome.tif"
# image_path = "/Users/swarchol/Research/exemplar-001/registration/exemplar-001.ome.tif"
image_path = "/Volumes/Simon/Greg/WD-76845-097.ome.tif"

# segmentation_path = "/mnt/c/Users/simon/Documents/Greg/for_simon/segmentation_mask/WD-76845-097.tif"
segmentation_path = "/Users/swarchol/Research/exemplar-001/segmentation/unmicst-exemplar-001/nuclei.ome.tif"

# csv_path = "/mnt/c/Users/simon/Documents/Greg/for_simon/unmicst-WD-76845-097_cellRing.csv"
csv_path = "/Users/swarchol/Research/exemplar-001/quantification/exemplar-001--unmicst_cell_umap_7_22.csv"

dest_path = "/Users/swarchol/Research/exemplar-001/cut"






In [66]:
!conda list | grep tifffile

tifffile                  2024.5.22          pyhd8ed1ab_0    conda-forge


In [65]:
# image_pyramid, multiscale = get_dask_arr(tf.TiffFile(image_path, is_ome=False))
# label_pyramid, _ = get_dask_arr(tf.TiffFile(segmentation_path, is_ome=False))
# df = pd.read_csv(csv_path)
img = (tf.TiffFile(image_path, is_ome=False))


AttributeError: 'TiffFile' object has no attribute 'size'

In [55]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")


DEBUG:root:test


In [56]:
for i in tqdm(range(len(img.pages))):
    sample_path = f"{dest_path}/channel_{i}"
    args = [image_path,
    segmentation_path,
    csv_path,
    sample_path,
    "-p", "8",  # Number of processes
        "--channels", (str(i+1)),  # Specific channels to include
        "--cache-size", str(1024 * 1024 * 1024 * 4),  # Cache size
    ]
    cellcutter.cli.cut(args)
    
    
# Now Create the Mask
sample_path = f"{dest_path}/mask"
args = [image_path,
segmentation_path,
csv_path,
sample_path,
"-p", "8",  # Number of processes
    "--channels", (str(i+1)),  # Specific channels to include
    "--cache-size", str(1024 * 1024 * 1024 * 4),  # Cache size
    "--mask-cells"
    
]
cellcutter.cli.cut(args)
# Remove the mask directory but keep mask_mask
import shutil
shutil.rmtree(f"{dest_path}/mask")
shutil.move(f"{dest_path}/mask_mask", f"{dest_path}/mask")


  0%|          | 0/12 [00:00<?, ?it/s]INFO:root:Namespace(p=8, z=False, force=False, window_size=None, mask_cells=False, channels=[1], cache_size=4294967296, chunk_size=32, cells_per_chunk=None, IMAGE='/Users/swarchol/Research/exemplar-001/registration/exemplar-001.ome.tif', SEGMENTATION_MASK='/Users/swarchol/Research/exemplar-001/segmentation/unmicst-exemplar-001/nuclei.ome.tif', CELL_DATA='/Users/swarchol/Research/exemplar-001/quantification/exemplar-001--unmicst_cell_umap_7_22.csv', DESTINATION='/Users/swarchol/Research/exemplar-001/cut/channel_0')
INFO:root:Loading cell data
INFO:root:Check if all cell IDs from the CSV are represented in the segmentation mask
INFO:root:Remove cells from segmentation mask that are not present in the CSV
INFO:root:Finding window size
INFO:root:Window size automatically set to (39, 39)
INFO:root:Found 11170 cells
INFO:root:Using chunks of shape (1, 11030, 39, 39)
INFO:root:Writing thumbnails to /Users/swarchol/Research/exemplar-001/cut/channel_0
INFO:

'/Users/swarchol/Research/exemplar-001/cut/mask'

In [57]:
# zarr_list = [zarr.open("{dest_path}/channel_{i}") for i in range(len(img.pages))]
# dest_path
paths_list = [f"{dest_path}/channel_{i}" for i in range(len(img.pages))]
zarr_list = [zarr.open(path) for path in paths_list]


In [58]:
combined_store = zarr.DirectoryStore(dest_path + "/combined")# [r.chunks for r in zarr_list]

In [59]:
combined = zarr.create(
            store=combined_store,
            overwrite=True,
            shape=(len(zarr_list), zarr_list[0].shape[1],zarr_list[0].shape[2],zarr_list[0].shape[3]),
            dtype=zarr_list[0].dtype,
            compressor=Blosc(cname="zstd", clevel=2, shuffle=Blosc.SHUFFLE)        )

In [60]:
for i in tqdm(range(len(zarr_list))):
    combined[i,:,:,:] = zarr_list[i][0]

100%|██████████| 12/12 [00:03<00:00,  3.14it/s]


In [61]:
# Remove the individual channel zarrs
for path in paths_list:
    shutil.rmtree(path)